In [21]:
import sys
import torch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sys.path.append('./models')
from models.matrix_a_linear import MatrixALinear

In [24]:
torch.set_printoptions(threshold=10_000)
def load_model(input_dim: int, output_dim: int):
    matrix_a_linear = MatrixALinear(input_dim=input_dim, output_dim=output_dim, bias=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    matrix_a_linear.load_state_dict(torch.load("../../saved_models/matrix_a.pkl", map_location=torch.device(device)))

    return matrix_a_linear

def save_model(new_model: MatrixALinear):
    torch.save(new_model.state_dict(),f'../../saved_models/matrix_a_after_pca.pkl')

In [25]:
model = load_model(100, 100)
# matrix_a_np = model.state_dict()['linear.weight'].numpy()
# print(model.linear.weight.data)
matrix_a_np = model.linear.weight.data.numpy()

In [84]:
matrix_a_np

array([[ 0.81704366, -0.2660322 , -0.08401775, ..., -0.30233875,
        -0.0488075 , -0.11722549],
       [-0.08853493,  0.61349964, -0.14548564, ..., -0.04823741,
         0.00391661, -0.01794411],
       [-0.2871844 , -0.2862975 ,  0.75088465, ..., -0.19625098,
        -0.1456132 , -0.18078662],
       ...,
       [-0.16676742, -0.07168139,  0.08625817, ...,  0.838528  ,
        -0.06899016, -0.19889902],
       [-0.01211979, -0.09629856,  0.16925535, ..., -0.2892095 ,
         1.0440856 , -0.07014164],
       [-0.13707556,  0.14942361,  0.14688483, ..., -0.25762913,
        -0.16823018,  1.0223261 ]], dtype=float32)

In [93]:
x = StandardScaler().fit_transform(matrix_a_np)
pca = PCA(n_components=10)
# pca = PCA(.95)
principalComponents = pca.fit_transform(x)


In [94]:
# principalComponents = StandardScaler().fit_transform(principalComponents)

In [95]:
pca.explained_variance_ratio_

array([0.49193215, 0.031477  , 0.02599666, 0.02435079, 0.0194992 ,
       0.01871691, 0.01654205, 0.01557036, 0.01453817, 0.01268661],
      dtype=float32)

In [96]:
new_weights = torch.from_numpy(principalComponents)

matrix_a_linear_after_pca = MatrixALinear(input_dim=10, output_dim=100, bias=False)
new_dict = {
    'linear.weight': new_weights
}
matrix_a_linear_after_pca.load_state_dict(new_dict)

<All keys matched successfully>

In [97]:
save_model(matrix_a_linear_after_pca)

In [98]:
matrix_a_linear_after_pca

MatrixALinear(
  (linear): Linear(in_features=10, out_features=100, bias=False)
)

In [91]:
inp = torch.rand((1,10))
print(inp.shape)
inp

torch.Size([1, 10])


tensor([[0.8076, 0.9599, 0.9586, 0.7393, 0.7662, 0.7674, 0.5864, 0.6255, 0.2695,
         0.5335]])

In [92]:
result = matrix_a_linear_after_pca(inp)
result

tensor([[-1.8410e+00, -7.4372e-01,  9.5965e-01,  2.0536e+00,  2.9057e-01,
          4.6662e+00,  5.4613e-01, -3.6284e+00,  1.2021e+00,  1.2461e+00,
          3.5131e+00, -2.3222e+00, -2.9401e+00,  8.9317e-01,  6.6089e-01,
         -3.8989e-01, -4.1775e-01, -1.5894e+00, -4.8385e-03,  3.5185e+00,
          4.5381e+00, -1.0057e+00,  1.6244e+00, -1.8851e+00, -2.1659e+00,
         -1.6307e-01, -1.4376e+00,  3.7083e+00,  1.3883e+00, -3.1221e+00,
          1.1177e+00, -3.7484e+00,  3.1693e+00,  1.7022e+00, -3.1291e-01,
         -2.8303e-01, -3.9888e+00,  3.9167e+00, -6.6794e-01, -5.4782e+00,
         -6.7741e-01, -7.6695e-01, -7.1392e-01, -3.3219e-01,  1.2018e+00,
          3.5794e+00, -1.7102e+00,  1.5329e+00, -4.2706e-01, -1.5176e+00,
         -5.2782e+00,  7.3752e-01,  2.4166e+00,  6.5877e+00, -1.2992e+00,
         -1.0325e+00,  3.9360e+00, -1.9155e+00,  1.1301e-01, -6.7083e-01,
         -2.0461e+00, -1.8880e+00,  2.6422e+00, -2.4532e+00,  9.4993e-01,
          2.2931e+00,  4.0740e+00, -3.